# **Optimization Model (Using Predicted Ingredient Requirements + Deterministic Purchasing)**

This model takes as **inputs**:

* Forecast pizza demand $d_p$
* Predicted **ingredient requirements** $\hat{x}_i$ (from your ML model)
* Starting inventory $I_i^0$

Using these, the system **automatically purchases only the missing amount**:

$
\text{purchase}_i = \max(\hat{x}_i - I_i^0, 0)
$

No purchasing decision variable is optimized — purchasing is deterministic and based on predictions.

The optimization then decides:

* How many pizzas to produce $(y_p)$
* How much demand to leave unmet $(z_p)$
* How much leftover inventory remains after production $(w_i)$

---

## **Sets**

* $P$: Set of pizza SKUs
* $I$: Set of ingredients

---

## **Parameters**

* $d_p$: Forecast demand for pizza (p)
* $\hat{x}_i$: Predicted ingredient **requirement** (kg)
* $I_i^0$: Starting inventory of ingredient (i) (kg)
* $\pi_p$: Sale price of pizza (p)
* $\alpha_p$: Penalty for unmet demand
* $r_{p,i}$: Ingredient usage (kg) of ingredient (i) in pizza (p)
* $s_i$: Spoilage cost per kg of leftover inventory
* $c_i$: Purchase cost per kg of ingredient (i)

---

## **Derived Quantities**

Before solving the optimization for each day, we compute:

### **Purchasing Rule**

$
\text{purchase}_i = \max(\hat{x}_i - I_i^0,, 0)
$

### **Available Inventory After Purchasing**

$
I_i^{\text{avail}} = I_i^0 + \text{purchase}_i
$

This is the **total usable inventory** for production.

---

## **Decision Variables**

* $y_p \ge 0$: Pizzas of type (p) produced
* $z_p \ge 0$: Unmet demand for pizza (p)
* $w_i \ge 0$: Leftover inventory of ingredient (i) after production

---

## **Objective Function**

The optimization maximizes:

* Profit from pizza production
* Minus penalties for unmet demand
* Minus spoilage cost
* Minus purchase cost of ingredients

$
\max \left[
\sum_{p\in P} \pi_p y_p ;-;
\sum_{p\in P} \alpha_p z_p ;-;
\sum_{i\in I} s_i w_i ;-;
\sum_{i\in I} c_i \cdot \text{purchase}_i
\right]
$

Note:
Purchase costs are **inputs**, not decision variables.

---

## **Constraints**

### **1. Demand Satisfaction**

$
y_p + z_p = d_p
\qquad \forall p \in P
$

---

### **2. Ingredient Availability**

Production cannot exceed available inventory:

$
\sum_{p\in P} r_{p,i}  y_p \le I_i^{\text{avail}}
\qquad \forall i \in I
$

---

### **3. Leftover Inventory Definition**

$
w_i = I_i^{\text{avail}} * \sum_{p\in P} r_{p,i} , y_p
  \qquad
  w_i \ge 0
  \qquad \forall i \in I
$

---

### **4. Nonnegativity / Integrality**

$
y_p \ge 0,\quad
z_p \ge 0,\quad
w_i \ge 0
$

In [1]:
using CSV
using DataFrames
using LinearAlgebra
using JuMP
using Gurobi

# Load datasets
daily_sales = CSV.read("modelling data/daily_sales.csv", DataFrame)
daily_sales_ingredients_predictions = CSV.read("modelling data/daily_sales_ingredients_predictions.csv", DataFrame)
daily_sales_ingredients_actuals = CSV.read("modelling data/daily_sales_ingredients_actuals.csv", DataFrame)
daily_sales_actuals = CSV.read("modelling data/daily_sales_actuals.csv", DataFrame)
ingredient_prices = CSV.read("modelling data/ingredient_prices.csv", DataFrame)
pizza_ingredient_matrix = CSV.read("modelling data/pizza_ingredient_matrix.csv", DataFrame)
pizzas = CSV.read("modelling data/pizzas.csv", DataFrame)

# Convert DataFrames to matrices if needed
daily_sales_mat = Matrix(daily_sales[:, Not(:date)])   # remove 'date' column
ingredient_prices_mat = Matrix(ingredient_prices[:, Not(:ingredient)]) # keep only prices
pizza_ingredient_matrix_mat = Matrix(pizza_ingredient_matrix[:, Not([:pizza_type_id, :pizza_name, :category])])
pizzas_mat = Matrix(pizzas[:, Not([:pizza_id, :pizza_type_id, :size])]) # only prices
;

In [2]:
# Get original column names
orig_cols = names(daily_sales_ingredients_predictions)

# Create cleaned names by removing the prefix where it appears
clean_cols = Symbol.(replace.(String.(orig_cols), "ingredient_" => ""))

# Apply new names back to the DataFrame
rename!(daily_sales_ingredients_predictions, Dict(orig_cols .=> clean_cols))
rename!(daily_sales_ingredients_actuals, Dict(orig_cols .=> clean_cols));

In [3]:
# --- Sets ---
P = pizzas.pizza_id                     # pizza SKUs
I = ingredient_prices.ingredient        # ingredients

# --- Parameters ---

# Pizza demand (single day forecast)
d = Dict(p => daily_sales_actuals[!, p][1] for p in P)

# Pizza sale price
π = Dict(p => pizzas[!, :price][findfirst(==(p), pizzas[!, :pizza_id])] for p in P)

# Predicted ingredient availability (x_i is FIXED, not a decision variable)
x_pred = Dict(i => daily_sales_ingredients_predictions[!, i][1] for i in I)

# Ingredient usage per pizza (kg): r[p, i]
r = Dict()
for p in P
    pizza_idx = findfirst(==(p), pizzas[!, :pizza_id])
    pizza_type_id = pizzas[!, :pizza_type_id][pizza_idx]

    for i in I
        row_idx = findfirst(==(pizza_type_id), pizza_ingredient_matrix[!, :pizza_type_id])
        r[(p,i)] = pizza_ingredient_matrix[row_idx, i]
    end
end

# Initial ingredient inventory
I0 = Dict(i => 0.0 for i in I)

# Penalty for unmet pizza demand
α = Dict(p => 10.0 for p in P)

# Spoilage cost per kg of leftover ingredient
s = Dict(i => 0.1 * ingredient_prices[!, :price_per_kg][findfirst(==(i), ingredient_prices[!, :ingredient])] 
            for i in I)

# Storage capacity
C = 200.0
;


In [4]:
function run_strategy(daily_sales_actuals, daily_supply_matrix)

    total_days = size(daily_sales_actuals, 1)

    metrics = DataFrame(
        day = 1:total_days,
        pizza_produced = zeros(total_days),
        unmet_pizza_demand = zeros(total_days),
        leftover_ingredients = zeros(total_days),
        daily_profit = zeros(total_days)
    )

    total_profit = 0.0

    # Initial leftover inventory (start with 0)
    I0_current = Dict(i => 0.0 for i in I)

    for t in 1:total_days

        # --- 1. Demand for pizzas today (always actual) ---
        d_day = Dict(p => daily_sales_actuals[!, p][t] for p in P)

        # --- 2. Predicted / actual ingredient *requirements* (not free supply) ---
        req_i = Dict(i => daily_supply_matrix[!, i][t] for i in I)

        # --- 3. Purchase only what you lack ---
        purchase_i = Dict(i => max(req_i[i] - I0_current[i], 0) for i in I)

        # --- 4. Available ingredients for production ---
        I_available = Dict(i => I0_current[i] + purchase_i[i] for i in I)

        # --- 5. Solve production optimization ---
        model = Model(Gurobi.Optimizer)
        set_silent(model)

        @variable(model, y[p in P] >= 0, Int)   # pizzas produced
        @variable(model, z[p in P] >= 0, Int)   # unmet demand
        @variable(model, w[i in I] >= 0)        # leftover ingredients AFTER production

        # Objective: profit = revenue - unmet_penalty - spoilage - purchase_cost
        @objective(model, Max,
            sum(π[p] * y[p] for p in P)
            - sum(α[p] * z[p] for p in P)
            - sum(s[i] * w[i] for i in I)
            - sum(10*s[i] * purchase_i[i] for i in I)
        )

        # Pizza demand constraint
        @constraint(model, [p in P], y[p] + z[p] == d_day[p])

        # Ingredient availability
        @constraint(model, [i in I],
            sum(r[(p,i)] * y[p] for p in P) <= I_available[i]
        )

        # Leftover definition
        @constraint(model, [i in I],
            w[i] == I_available[i] - sum(r[(p,i)] * y[p] for p in P)
        )

        optimize!(model)

        # Extract decisions
        y_opt = Dict(p => round(Int, value(y[p])) for p in P)
        z_opt = Dict(p => round(Int, value(z[p])) for p in P)
        w_opt = Dict(i => value(w[i]) for i in I)

        # Inventory rolls over to next day
        I0_current = deepcopy(w_opt)

        # Daily profit
        daily_profit = objective_value(model)
        total_profit += daily_profit

        # Save metrics
        metrics.pizza_produced[t] = sum(values(y_opt))
        metrics.unmet_pizza_demand[t] = sum(values(z_opt))
        metrics.leftover_ingredients[t] = sum(values(w_opt))
        metrics.daily_profit[t] = daily_profit
    end

    # Final spoilage cost (optional)
    final_spoilage_cost = sum(s[i] * I0_current[i] for i in I)
    total_profit_net = total_profit - final_spoilage_cost

    return (total_profit = total_profit_net, metrics = metrics)
end


# --- Run predictive strategy (use predicted supplies) ---
pred_result = run_strategy(daily_sales_actuals, daily_sales_ingredients_predictions)

# --- Run wait-and-see strategy (use actual supplies) ---
ws_result   = run_strategy(daily_sales_actuals, daily_sales_ingredients_actuals)

println("Predictive total profit (net): ", pred_result.total_profit)
println("Wait-and-See total profit (net): ", ws_result.total_profit)


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20